<a href="https://colab.research.google.com/github/its-rajesh/Music-Speech-Separation/blob/main/MSClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MUSIC SPEECH CLASSIFIER**

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
import soundfile as sf
import librosa as lb
import numpy as np
from matplotlib import pyplot as plt
import os
import pandas as pd

### Reading STFT & Chroma STFT Features

In [3]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [7]:
path = '/drive/My Drive/Projects/Music-Speech-Separation/Dataset/'

In [8]:
folders = os.listdir(path)
folders

['music', 'Mixture', 'Overlay', 'speech']

In [9]:
audiofiles = []
for folder in folders:
    files = os.listdir(path+'/'+folder)
    files = sorted(files)
    audio = []
    for file in files:
        data, samplerate = lb.load(path+'/'+folder+'/'+file, sr=8000, mono=True)
        audio.append(data)
    audiofiles.append(audio)

In [10]:
music = np.array(audiofiles[0])
mixture = np.array(audiofiles[1])
overlay = np.array(audiofiles[2])
speech = np.array(audiofiles[3])

dataset = [music, mixture, overlay, speech]

In [65]:
music[0], len(music[0]), len(music[0])/8000

(array([-0.06211764, -0.17527802, -0.13408872, ...,  0.06697536,
         0.07972796,  0.11764227], dtype=float32), 120000, 15.0)

In [75]:
def createchunks(audio):
  start = 0
  stop = 4000 # for 0.5 seconds (8Khz): 1 sec has 8000 samples.
  chunks = []
  for i in range(len(audio)//4000):
    chunks.append(audio[start:stop])
    start = stop
    stop = start+4000

  return chunks

#x = np.array(createchunks(music[0]))
#print(x.shape)

In [110]:
''' #SAMPLE FOR CHUNKS STFT
chunks = np.array(createchunks(music[0]))
spect = []
for i in chunks:
  spect.append(np.abs(lb.stft(i,n_fft=512))) #window length = nfft and hop length = win length //4

spect = np.array(spect)
spect.shape
'''


' #SAMPLE FOR CHUNKS STFT\nchunks = np.array(createchunks(music[0]))\nspect = []\nfor i in chunks:\n  spect.append(np.abs(lb.stft(i,n_fft=512))) #window length = nfft and hop length = win length //4\n\nspect = np.array(spect)\nspect.shape\n'

##### STFT Features

In [105]:
datasetstft = []
index = 0
y, y_train = [], []
for data in [music, speech]:
    spect = []
    for audio in data:
      chunks = np.array(createchunks(audio))
      chunks_spect = []
      for i in chunks:
        chunks_spect.append(np.abs(lb.stft(i,n_fft=512))) #window length = nfft and hop length = win length //4

      chunks_spect = np.array(chunks_spect)

      spect.append(chunks_spect)

      if index == 0:
        y.append(np.zeros(30))
      else:
        y.append(np.ones(30))
    
    index += 1
    y_train.append(np.array(y))
    datasetstft.append(np.array(spect))

In [108]:
y_train[0].shape, len(y_train)

((50, 30), 2)

In [102]:
datasetstft[0].shape, len(datasetstft)

((50, 30, 257, 32), 2)

##### Chroma Features

In [109]:
datasetchroma = []
for data in [music, speech]:
    chroma_spect = []
    for audio in data:
      chunks = np.array(createchunks(audio))
      chunks_chroma = []
      for i in chunks:
        chunks_chroma.append(lb.feature.chroma_stft(y=audio)) 

      chunks_chroma = np.array(chunks_chroma)

      chroma_spect.append(chunks_chroma)
    datasetchroma.append(np.array(chroma_spect))

##### Reshaping the training datasets

## CNN MODEL



> Indented block



In [33]:
import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,Flatten,Dropout,Conv2D,MaxPool2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [118]:
batch_size = 32
epochs = 10
inp = Input(shape =(257, 32, 1))
out_dim = 2

In [119]:
start = time.time()

x = Conv2D(32, 3, activation='relu')(inp)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.30)(x)

x = Conv2D(64, 3, activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.30)(x)

x = Flatten()(x)
x = Dense(100, activation='relu')(x)
x = Dropout(0.30)(x)

x = Dense(out_dim, activation='softmax')(x)

In [121]:
x_train = np.array(datasetstft, dtype=object)
y_train = np.array(y_train, dtype=object)

In [122]:
classify = Model(inputs = inp, outputs = x)
classify.compile(loss='SparseCategoricalCrossentropy', optimizer = 'adam', metrics='accuracy') 
#classify.summary()
classify_train = classify.fit(x_train,y_train, epochs = epochs ,batch_size=batch_size)
end = time.time()
print("time taken",time)

ValueError: ignored